# Aprendizaje automático (Machine Learning)

### Inteligencia Artificial
### Universidad de Sevilla

**Scikit-Learn**, abreviado como `sklearn`, se ha establecido como una piedra angular en el ecosistema de herramientas de aprendizaje automático en Python. Este paquete open-source, desarrollado por un conjunto de expertos en aprendizaje automático, proporciona una interfaz simple y eficiente para la implementación de algoritmos de aprendizaje supervisado y no supervisado, así como herramientas para evaluar y ajustar modelos. Desde su inicio, sklearn ha sido reconocido por su enfoque coherente y su énfasis en la usabilidad, haciendo que la construcción y experimentación con modelos de aprendizaje automático sea accesible para desarrolladores y científicos de datos por igual.

Para ilustrar el concepto de aprendizaje supervisado vamos a usar el conjunto de datos [_Car Evaluation_](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation) del repositorio [UCI](http://archive.ics.uci.edu/ml/). Este conjunto de datos contiene información acerca de la idoneidad de una serie de coches, en función de los siguientes atributos discretos:
* _buying_: precio de compra. Posibles valores: `vhigh`, `high`, `med`, `low`.
* _maint_: coste de mantenimiento. Posibles valores: `vhigh`, `high`, `med`, `low`.
* _doors_: número de puertas. Posibles valores: `2`, `3`, `4`, `5more`.
* _persons_: número de asientos. Posibles valores: `2`, `4`, `more`.
* _lug\_boot_: tamaño del maletero. Posibles valores: `small`, `med`, `big`.
* _safety_: nivel de seguridad estimada. Posibles valores: `low`, `med`, `high`.

La idoneidad de cada coche se indica mediante _acceptability_, que los clasifica como `unacc`, `acc`, `good` o `vgood`. Este será nuestro objetivo.

Para leer los datos desde el fichero `cars.csv` que se proporciona se pueden evaluar las siguientes expresiones ([_Pandas_](http://pandas.pydata.org/) y [_NumPy_](http://www.numpy.org/) son paquetes de _Python_ para análisis de datos y cálculo científico, respectivamente):

In [1]:
import pandas
import numpy

In [2]:
cars = pandas.read_csv('cars.csv', header=None,
                       names=['buying', 'maint', 'doors', 'persons',
                              'lug_boot', 'safety', 'acceptability'])

# En general, si no se dice nada siempre se va a referir a las columnas, como en names.
# Si el archivo no se encuentra en la misma carpeta, se pone como /data/cars.csv por ej.

In [3]:
# Número de filas y columnas
# dim (tabla) en R

print(cars.shape)

(1728, 7)


In [4]:
# Muestra las 10 primeras filas
# head (tabla) en R. 
cars.head(10)

# RECORDAR que en python se empieza desde 0. 

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


_sklearn_ no puede trabajar directamente con el conjunto de datos anterior, ya que asume que los valores de las variables discretas están codificadas con números enteros. Para transformar los datos a un formato adecuado ofrece diversas operaciones de preprocesamiento, entre las que se encuentran `OrdinalEncoder`, para codificar los atributos discretos a numéricos, y `LabelEncoder`, para codificar la variable objetivo de discreta a númerico.

Existen muchas codificadores, hay que encontrar el adecuado para tus datos, por ejemplo para procesamiento de texto natural se suele utilizar `CountVectorizer()`.


In [5]:
from sklearn import preprocessing

In [6]:
atributos = cars.loc[:, 'buying':'safety']  # selección de las columnas de atributos
# : todas las filas, y todas las columnas que vayan desde buying hasta safety. Accedemos a un número de filas y
# columnas en concreto. 
# . loc es porque le estoy dando una localización concreta, como si le doy unas coordenadas. 

objetivo = cars['acceptability']  # selección de la columna objetivo, debe ser el resultado de la clasificacion

# en R era cars[["acceptability"]] para entrar en una columna entera. 

In [7]:
# Para realizar una codificación de los datos, se crea una instancia del tipo de
# codificación pretendida y se ajusta a los datos disponibles mediante el método fit.

#Primero crear la instancia del modelo y luego se entrena. 

# El codificador adecuado para los atributos es OrdinalEncoder, ya que permite
# trabajar con el array completo de valores de los atributos.
codificador_atributos = preprocessing.OrdinalEncoder() # Codificar.
codificador_atributos.fit(atributos) # Lo entreno para que sea capaz de clasificar los atributos que le doy. 

# Categorías detectadas por el codificador para cada atributo
print(codificador_atributos.categories_)
# cada una de las clasificaciones las pilla y las pasa a números. 

[array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['2', '3', '4', '5more'], dtype=object), array(['2', '4', 'more'], dtype=object), array(['big', 'med', 'small'], dtype=object), array(['high', 'low', 'med'], dtype=object)]


In [8]:
# Una vez ajustado el codificador, el método transform permite codificar los
# valores de los atributos
atributos_codificados = codificador_atributos.transform(atributos)
print(atributos_codificados)

[[3. 3. 0. 0. 2. 1.]
 [3. 3. 0. 0. 2. 2.]
 [3. 3. 0. 0. 2. 0.]
 ...
 [1. 1. 3. 2. 0. 1.]
 [1. 1. 3. 2. 0. 2.]
 [1. 1. 3. 2. 0. 0.]]


In [9]:
# El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
# con una lista o array unidimensional de sus valores
codificador_objetivo = preprocessing.LabelEncoder()

# El método fit_transform ajusta la codificación y la aplica a los datos justo
# a continuación
objetivo_codificado = codificador_objetivo.fit_transform(objetivo)

Una vez codificadas las variables, es necesario separar el conjunto de datos en dos: un conjunto de entrenamiento, que se usará para construir los distintos modelos; y un conjunto de prueba, que se usará para comparar los distintos modelos.

Un detalle a tener en cuenta es que la distribución de ejemplos en las distintas clases de aceptabilidad no es uniforme: hay 1210 coches (un 70.023&nbsp;% del total) clasificados como inaceptables (`unacc`), 384 coches (22.222&nbsp;%) clasificados como aceptables (`acc`), 69 coches (3.993&nbsp;%) clasificados como buenos (`good`) y 65 coches (3.762&nbsp;%) clasificados como muy buenos (`vgood`).

In [10]:
# Frecuencia total de cada clase de aceptabilidad
print(pandas.Series(objetivo).value_counts(normalize=True))

# panda y numpys son paquetes muy grandes. Solo nos pide que aprendamos a leer tablas y a usar los datos. 
# para ver la freq total. 
# con pandas accedemos a nuestra variable de objetivo, accedo a esa columna y sigue siendo una tabla de una
# sola cloumna, no como en R que se convertía en R. 

# value_counts es como el table de R -> freq abs dentro de una tabla. 
# Combinando estas dos funciones nos da la freq rel.
# las clasificaciones me las da el series y las freq abs me las da el value_counts. 
# Ya sabemos que esta es la proporción que tenemos que seguir en nuestros conjuntos de datos. 


acceptability
unacc    0.700231
acc      0.222222
good     0.039931
vgood    0.037616
Name: proportion, dtype: float64


Es conveniente, por tanto, que la separación de los ejemplos se realice de manera estratificada, es decir, intentando mantener la proporción anterior tanto en el conjunto de entrenamiento como en el de prueba.

Para dividir un conjunto de datos en un subconjunto de entrenamiento y otro de prueba, _sklearn_ proporciona la función `train_test_split`.

In [11]:
from sklearn import model_selection

# Esto sirve para separar los conjuntos de datos manteniendo las proporciones. 

In [14]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_codificados, objetivo_codificado,
        # Valor de la semilla aleatoria, para que el muestreo sea reproducible,
        # a pesar de ser aleatorio
        # Muestreo aleatorio dentro de nuestro data set. Normalmente se pone una semilla súper alta. 
        random_state=12345,
        # Tamaño del conjunto de prueba, en cuanto queremos dividir nuestro data set original.
        # Le estoy diciendo que quiero que el 33% me lo guarde para pruebas. 
        test_size=.33,
        # Estratificamos respecto a la distribución de valores en la variable objetivo
        stratify=objetivo_codificado) # Siguiendo siempre estas proporciones. 

In [17]:
# Comprobamos que el conjunto de prueba contiene el 33 % de los datos, en la misma proporción
# con respecto a la variable objetivo
print('Cantidad de ejemplos de pruebas requeridos:', 1728 * .33)
print('Filas del array de atributos de prueba:', atributos_prueba.shape[0])
print('Longitud del vector de objetivos de prueba:', len(objetivo_prueba))
print('Proporción de clases en el vector de objetivos de prueba:')
print(pandas.Series(
        codificador_objetivo.inverse_transform(objetivo_prueba)
      ).value_counts(normalize=True))

Cantidad de ejemplos de pruebas requeridos: 570.24
Filas del array de atributos de prueba: 571
Longitud del vector de objetivos de prueba: 571
Proporción de clases en el vector de objetivos de prueba:
unacc    0.700525
acc      0.222417
good     0.040280
vgood    0.036778
Name: proportion, dtype: float64


In [18]:
# Comprobamos que el conjunto de entrenamiento contiene el resto de los datos, en la misma
# proporción con respecto a la variable objetivo
print('Cantidad de ejemplos de entrenamiento requeridos:', 1728 * .67)
print('Filas del array de atributos de entrenamiento:', atributos_entrenamiento.shape[0])
print('Longitud del vector de objetivos de entrenamiento:', len(objetivo_entrenamiento))
print('Proporción de clases en el vector de objetivos de entrenamiento:')
print(pandas.Series(
        codificador_objetivo.inverse_transform(objetivo_entrenamiento)
      ).value_counts(normalize=True))

Cantidad de ejemplos de entrenamiento requeridos: 1157.76
Filas del array de atributos de entrenamiento: 1157
Longitud del vector de objetivos de entrenamiento: 1157
Proporción de clases en el vector de objetivos de entrenamiento:
unacc    0.700086
acc      0.222126
good     0.039758
vgood    0.038029
Name: proportion, dtype: float64


Para realizar aprendizaje supervisado en _sklearn_, basta crear una instancia de la clase de objetos que implemente el modelo que se quiera utilizar (_naive_ Bayes, árboles de decisión, _kNN_, etc.).

Cada una de estas instancias dispondrá de los siguientes métodos:
* El método `fit` permite entrenar el modelo, dados __por separado__ el conjunto de ejemplos de entrenamiento y la clase de cada uno de estos ejemplos.
* El método `predict` permite clasificar un nuevo ejemplo una vez entrenado el modelo.
* El método `score` calcula el rendimiento del modelo, dados __por separado__ el conjunto de ejemplos de prueba y la clase de cada uno de estos ejemplos.

### _Naive_ Bayes

_sklearn_ implementa _naive_ Bayes para atributos categóricos mediante instancias de la clase `CategoricalNB`. Para otro tipo de tareas, como la que se presentá más adelante con procesamiento del lenguaje natural, se usa MultinomialNB.

Dentro de tu data set cuenta las posiciones y busca. Solo se basa en las estadísticas. Tampoco hace falta saber el trasfondo del modelo pero no viene mal por algún error. 

Lo dificil de explorar el modelo: tenemos que saber como funcions se modelo. Si queremos ...
si usamos otro algoritmo, como el de kn vecinos, tnemeos qeu poner auqui e,l número de vecinos. 


In [19]:
from sklearn import naive_bayes

In [20]:
clasif_NB = naive_bayes.CategoricalNB(alpha=1.0) # categorical porque estamos trabajando para la categorización. 
clasif_NB.fit(atributos_entrenamiento, objetivo_entrenamiento) # entrenamos con fit,
# con score vemos el porcentaje de acierto. 
PARA EL EXAMEN - 0,70
# Podemos o mejorar el modelo (versión del modelo, como el nombre de ese tio raro o ....)

CategoricalNB()

El método `score` la tasa de acierto (_accuracy_) sobre un conjunto de datos de prueba. 

In [21]:
clasif_NB.score(atributos_prueba, objetivo_prueba)

0.8441330998248686

El método `predict` devuelve la clase predicha por el modelo para usarlo en un futuro con un nuevo ejemplo.